In [123]:
# %%% CVXPY %%%
import cvxpy
import numpy
# Construct the problem.
U = numpy.array([[1,4],
                 [0,4]])
V = numpy.array([[4,6],
                 [2,5]])
F = numpy.array([[6,56],
                 [20,8]])
la0 = [0.5,0.75]
laf0 = [0.3,0.9]
lamin = [-1]
lamax = [1]
lafmin = [0]
lafmax = [1]
la = cvxpy.Variable(2) # n = length of λ: 18121 
laf = cvxpy.Variable(2) # n = length of λf: 4318
objective = cvxpy.Minimize(cvxpy.sum_squares((la - la0)/la0) + cvxpy.sum_squares((laf-laf0)/laf0))
constraints = [lamin <= la, la <= lamax, lafmin <= laf, laf <= lafmax, la * (U - V) + laf * F == 0]
prob = cvxpy.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()
# The optimal value for λ is stored in `λ.value`.
print(la.value)
print(laf.value)
# The optimal Lagrange multiplier for a constraint is stored in
# `constraint.dual_value`.
print(constraints[0].dual_value) # 0 to 4

[0.5452479  0.80760463]
[0.01115148 0.15920221]
[0. 0.]


In [124]:
la.value @ U + laf.value @ F

array([3.79620088, 7.30951057])

In [125]:
la.value @ V

array([3.79620088, 7.30951057])

In [126]:
# %%% SCIPY %%%
from scipy.optimize import minimize
import scipy.optimize
import numpy
U = numpy.array([[1,4],
                 [0,4]])
V = numpy.array([[4,6],
                 [2,5]])
F = numpy.array([[6,56],
                 [20,8]])
la0 = [0.5, 0.75]
laf0 = [0.3, 0.9]
x0_m = numpy.array([la0,laf0])
x0 = x0_m.ravel()
def objective (x):
    return numpy.sum(((x[0:2] - x0[0:2])/x0[0:2])**2) + numpy.sum(((x[2:4] - x0[2:4])/x0[2:4])**2)
def constraint1(x):
    return x[0:2] @ (U - V) + x[2:4] @ F
b = (-1,1)
bnds = (b, b, b, b)
con1 = {'type':'eq', 'fun': constraint1}
sol = minimize(objective, x0, method = 'SLSQP', bounds = bnds, constraints = con1)
print(sol)

     fun: 1.6186350847319695
     jac: array([ 0.36198328,  0.20481646, -6.41885588, -1.82913035])
 message: 'Optimization terminated successfully.'
    nfev: 33
     nit: 5
    njev: 5
  status: 0
 success: True
       x: array([0.5452479 , 0.80760462, 0.01115148, 0.1592022 ])


In [171]:
# %%% PYOMO %%%
from pyomo.environ import *
import pyomo as po
opt = SolverFactory('ipopt')

#Construct the problem
model = ConcreteModel()
U = {}
U['a','a'] = 1
U['a','b'] = 4
U['b','a'] = 0
U['b','b'] = 4
V = {}
V['a','a'] = 4
V['a','b'] = 6
V['b','a'] = 2
V['b','b'] = 5
F = {}
F['a','a'] = 6
F['a','b'] = 56
F['b','a'] = 20
F['b','b'] = 8
model.content = Set(initialize = list('ab'))
model.row = Set(initialize = list('A'))
model.U = Param(model.content, model.content, initialize=U)
model.V = Param(model.content, model.content, initialize=V)
model.F = Param(model.content, model.content, initialize=F)
la0 = {}
la0['A', 'a'] = 0.5
la0['A', 'b'] = 0.75
laf0 = {}
laf0['A', 'a'] = 0.3
laf0['A', 'b'] = 0.9
model.la0 = Param(model.row, model.content, initialize=la0) #, default = 0 include when sparse
model.laf0 = Param(model.row, model.content, initialize=laf0)
model.la = Var(model.row, model.content, bounds = (-1, 1))
model.laf = Var(model.row, model.content, bounds = (-1, 1))


model.obj = Objective(expr = sum(((model.la[r, c] - model.la0[r, c])/model.la0[r, c])**2 + \
                      ((model.laf[r, c] - model.laf0[r, c])/model.laf0[r, c])**2 \
                                 for r in model.row for c in model.content), sense = minimize)


model.c1 = Constraint(expr = model.la[('A','a')]*(model.U[('a','a')]-model.V[('a','a')])+model.la[('A','b')]*(model.U[('b','a')]-model.V[('b','a')])+\
model.la[('A','a')]*(model.U[('a','b')]-model.V[('a','b')])+model.la[('A','b')]*(model.U[('b','b')]-model.V[('b','b')])+\
model.laf[('A','a')]*model.F[('a','a')]+model.laf[('A','b')]*model.F[('b','a')]+model.laf[('A','a')]*model.F[('a','b')]+\
model.laf[('A','b')]*model.F[('b','b')] == 0)

status = opt.solve(model)
status

{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 1, 'Number of variables': 4, 'Sense': 'unknown'}], 'Solver': [{'Status': 'ok', 'Message': 'Ipopt 3.11.1\\x3a Optimal Solution Found', 'Termination condition': 'optimal', 'Id': 0, 'Error rc': 0, 'Time': 0.0438845157623291}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [177]:
#Results
model.display()

Model unknown

  Variables:
    la : Size=2, Index=la_index
        Key        : Lower : Value              : Upper : Fixed : Stale : Domain
        ('A', 'a') :    -1 : 0.5491906526673481 :     1 : False : False :  Reals
        ('A', 'b') :    -1 : 0.8164073782649921 :     1 : False : False :  Reals
    laf : Size=2, Index=laf_index
        Key        : Lower : Value               : Upper : Fixed : Stale : Domain
        ('A', 'a') :    -1 :  0.0804129242736012 :     1 : False : False :  Reals
        ('A', 'b') :    -1 : 0.00748478904173008 :     1 : False : False :  Reals

  Objectives:
    obj : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 1.5367160042829249

  Constraints:
    c1 : Size=1
        Key  : Lower : Body                   : Upper
        None :   0.0 : -2.498001805406602e-16 :   0.0


In [178]:
#Check if constraint fullfilled
model.c1.pprint()

c1 : Size=1, Index=None, Active=True
    Key  : Lower : Body                                                                                               : Upper : Active
    None :   0.0 : -3*la[A,a] - 2*la[A,b] - 2*la[A,a] - la[A,b] + 6*laf[A,a] + 20*laf[A,b] + 56*laf[A,a] + 8*laf[A,b] :   0.0 :   True
